In [ ]:
import os
import tensorflow as tf
from IPython.display import YouTubeVideo


frame_dir = 'train/'
dataset = dict()
overall_dataset = []
max_features_vector = []

print(os.listdir(frame_dir))
files=os.listdir(frame_dir)

i=0

for file in files:
         
    frame_lvl_record = frame_dir + file
    for example in tf.python_io.tf_record_iterator(frame_lvl_record):

        i=i+1
        
        tf_example = tf.train.Example.FromString(example)
        tf_seq_example = tf.train.SequenceExample.FromString(example)

        n_frames = len(tf_seq_example.feature_lists.feature_list['audio'].feature)
        
        max_features_vector.append(n_frames)

        vid_id = tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8')
        vid_labels = tf_example.features.feature['labels'].int64_list.value    

        rgb_values = []
        audio_values = []
        for i in range(n_frames):
            rgb_values.append(tf_seq_example.feature_lists.feature_list['rgb']
                            .feature[i].bytes_list.value[0])
            audio_values.append(tf_seq_example.feature_lists.feature_list['audio']
                            .feature[i].bytes_list.value[0])
        sess = tf.InteractiveSession()
        rgb_frame = tf.decode_raw(rgb_values, tf.uint8).eval()
        audio_frame = tf.decode_raw(audio_values, tf.uint8).eval()
        sess.close()
        tf.reset_default_graph()

        dataset['id'] = vid_id
        dataset['labels'] = list(vid_labels)
        dataset['rgb_frame'] = list(rgb_frame)
        dataset['audio_frame'] = list(audio_frame)
        overall_dataset.append(dataset)
        
        
print(max_features_vector)
print(overall_dataset)



['traincS.tfrecord', 'trainIc.tfrecord', 'trainpj.tfrecord', 'trainWB.tfrecord']
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
[140, 300, 300, 186, 244, 162, 300, 287, 300, 197, 214, 300, 177, 251, 300, 298, 181, 199, 150, 164, 239, 300, 200, 237, 180, 158, 286, 297, 196, 182, 299, 299, 300, 300, 165, 151, 192, 219, 166, 228, 300, 218, 288, 185, 228, 240, 178, 232, 181, 300, 141, 258, 232, 238, 258, 286, 300, 135, 277, 300, 272, 132, 191, 142, 296, 300, 300, 209, 177, 300, 223, 300, 300, 300, 229, 273, 300, 209, 216, 255, 300, 271, 136, 300, 145, 129, 300, 250, 300, 300, 300, 300, 183, 277, 300, 300, 300, 300, 300, 300, 300, 211, 211, 183, 300, 300, 259, 134, 179, 209, 126, 201, 294, 222, 264, 300, 206, 276, 300, 300, 288, 291, 166, 300, 171, 263, 218, 263, 288, 300, 300, 300, 248, 300, 300, 126, 262, 300, 141, 224, 299, 285, 160, 181, 172, 300, 300, 154, 191, 300, 266, 300, 271, 256, 177, 283, 167, 177, 300, 218, 137, 222, 300, 167, 199, 210, 232

In [15]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten

model = Sequential()


model.add(Conv2D(64, 3, 3, activation='relu', input_shape=(3800,15,2)))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())


C:\Users\BUSINESS NETWORK\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", input_shape=(3800, 15,...)`
  import sys


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 3798, 13, 64)      1216      
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 3796, 11, 32)      18464     
_________________________________________________________________
flatten_5 (Flatten)          (None, 1336192)           0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                13361930  
Total params: 13,381,610
Trainable params: 13,381,610
Non-trainable params: 0
_________________________________________________________________
None
